In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
# import numpy as np

In [3]:
database_type = "/training"

#### Interpolating Data across patient timeseries

In [4]:
finalMerge = pd.read_csv('../eICU' + database_type + '/finalMerge.csv')
finalMerge

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
0,141765,1179.0,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141765,189.0,NaN,76.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN
2,141765,1169.0,NaN,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,141765,1534.0,NaN,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,141765,1164.0,NaN,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1682237,3139532,7081.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1682238,3139532,9847.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1682239,3139532,5656.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1682240,3158501,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [5]:
finalMerge.loc[(finalMerge['patientunitstayid'] == 346380)].sort_values(by='observationoffset')

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
1647664,346380,34.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
1677078,346380,194.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1677077,346380,481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1677074,346380,685.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1677076,346380,2255.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1677079,346380,2310.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1677073,346380,5028.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1677075,346380,5035.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [6]:
idList = sorted(set(list(finalMerge['patientunitstayid'])))
finalData = pd.DataFrame()

In [36]:
for id in idList[:51]:
    df = finalMerge[finalMerge['patientunitstayid'] == id]
    sepsisDiagnosisTime = df[df['diagnosis'] == 1].observationoffset
    if len(sepsisDiagnosisTime) != 0:
        print(sepsisDiagnosisTime)
#     df = df[df['observationoffset'].le(sepsisDiagnosisTime)]
#     df.loc[:, 'temperature'] = df['temperature'].interpolate(method='linear', limit_direction='both')
#     df.loc[:, 'heartrate'] = df['heartrate'].interpolate(method='linear', limit_direction='both')
#     df.loc[:, 'respiration'] = df['respiration'].interpolate(method='linear', limit_direction='both')
#     df.loc[:, 'systemicsystolic'] = df['systemicsystolic'].interpolate(method='linear', limit_direction='both')
#     df.loc[:, 'creatinine'] = df['creatinine'].interpolate(method='linear', limit_direction='both')
#     df.loc[:, 'wbcx1000'] = df['wbcx1000'].interpolate(method='linear', limit_direction='both')
#     df.loc[:, 'lactate'] = df['lactate'].interpolate(method='linear', limit_direction='both')
#     df.loc[:, 'urineoutputbyweight'] = df['urineoutputbyweight'].interpolate(method='linear', limit_direction='both')
#     df.loc[:, 'diagnosis'] = df['diagnosis'].interpolate(method='linear')
#     df['diagnosis'].fillna('0', inplace=True)
#     finalData = finalData.append(df)

finalData

1678032    330.0
1678035    167.0
Name: observationoffset, dtype: float64
1679388    219.0
1679390     39.0
1679392    685.0
Name: observationoffset, dtype: float64
1682155    51.0
Name: observationoffset, dtype: float64
1680252    10.0
Name: observationoffset, dtype: float64


""


In [8]:
# path = "../eICU/training/"
# finalData.to_csv(path + "finalData.csv", sep=',', index=False, encoding='utf-8')
